In [7]:
import sys
sys.path.append('..')

import os
import pandas as pd
import numpy as np
import re
import numpy as np
from gensim.models import KeyedVectors
from konlpy.tag import Twitter

In [8]:
with open('../semnet/wordlist.txt', 'r') as f:
    raw = f.read()
    wordlist = raw.split()

In [9]:
pretrained_embeddings = '../../model_weights/pos.vec'
pos_vectors = KeyedVectors.load_word2vec_format(pretrained_embeddings, binary=False)

In [18]:
twitter = Twitter()

def normalize(array):
    norm = np.linalg.norm(array)
    return array / norm

def create_word_vector(word):
    pos_list = twitter.pos(word, norm=True)
    word_vector = np.sum([pos_vectors.word_vec(str(pos).replace(" ", "")) for pos in pos_list], axis=0)
    return normalize(word_vector)

In [20]:
vec = create_word_vector('강아지')

In [21]:
vec.shape

(200,)

In [22]:
from gensim.similarities.index import AnnoyIndexer

In [24]:
indexer = AnnoyIndexer(pos_vectors, 2)

In [30]:
pos_list = twitter.pos('강아지', norm=True)

In [38]:
query = str(pos_list[0])

In [42]:
query = query.replace(" ", "")

In [45]:
pos_vectors.most_similar(query, topn=10, indexer=indexer)

[(&quot;(&#39;강아지&#39;,&#39;Noun&#39;)&quot;, 1.0),
 (&quot;(&#39;반려견&#39;,&#39;Noun&#39;)&quot;, 0.6310654580593109),
 (&quot;(&#39;고양이&#39;,&#39;Noun&#39;)&quot;, 0.6296913921833038),
 (&quot;(&#39;동물병원&#39;,&#39;Noun&#39;)&quot;, 0.5350783169269562),
 (&quot;(&#39;애견&#39;,&#39;Noun&#39;)&quot;, 0.5251836776733398),
 (&quot;(&#39;원숭이&#39;,&#39;Noun&#39;)&quot;, 0.5181112289428711),
 (&quot;(&#39;수의사&#39;,&#39;Noun&#39;)&quot;, 0.51172935962677),
 (&quot;(&#39;유기동물&#39;,&#39;Noun&#39;)&quot;, 0.505106121301651),
 (&quot;(&#39;동물보호법&#39;,&#39;Noun&#39;)&quot;, 0.5039792656898499),
 (&quot;(&#39;애완&#39;,&#39;Noun&#39;)&quot;, 0.4968421459197998)]

In [58]:
def get_pruned_vectors():
    ''' Saves a new file of keyed embeddings corresponding to only
        the words in the basic word list.
    '''
    with open('../semnet/wordlist.txt', 'r') as f:
        raw = f.read()
        basic_words = raw.split()
    
    with open('../semnet/basic_word_embeddings.vec', 'w') as f:
        for word in basic_words:
            try:
                vec = create_word_vector(word)
                f.write('{} {}\n'.format(word, ' '.join(vec.astype('str'))))
            except KeyError:
                print('word not found: {}'.format(word))

get_pruned_vectors()

word not found: 걸레
word not found: 도너츠
word not found: 말(말씀)
word not found: 메론
word not found: 무(채소)
word not found: 비누방울
word not found: 시소
word not found: 이(이빨)
word not found: 일(하다)
word not found: 잠바
word not found: 쟁반
word not found: 차(자동차)
word not found: 참새
word not found: 크레파스
word not found: 다르다
word not found: 모르다
word not found: 알다
word not found: 놀라다
word not found: 씻다
word not found: 박수치다
word not found: 배부르다
word not found: 웃다
word not found: 입다
word not found: 다니다
word not found: 뛰다
word not found: 바르다
word not found: 살다
word not found: 들다
word not found: 마시다
word not found: 안되다
word not found: 울다
word not found: 기다리다
word not found: 끄다
word not found: 넣다
word not found: 눕다
word not found: 닫다
word not found: 던지다
word not found: 때리다
word not found: 만들다
word not found: 묻다
word not found: 벗다
word not found: 빗다
word not found: 신다
word not found: 꺼내다
word not found: 넘어지다
word not found: 멈추다
word not found: 빨다
word not found: 쓰다(모자)
word not found: 앉다
word not found: 춤추다
word

In [60]:
basic_vectors = KeyedVectors.load_word2vec_format('../semnet/basic_word_embeddings.vec', binary=False)

In [67]:
query = '엄마'
basic_vectors.most_similar(positive=query, topn=10)

[(&#39;아빠&#39;, 0.7519980669021606),
 (&#39;딸&#39;, 0.6410537958145142),
 (&#39;언니&#39;, 0.6316390037536621),
 (&#39;오빠&#39;, 0.6312486529350281),
 (&#39;아기&#39;, 0.5952275991439819),
 (&#39;선생님&#39;, 0.5938552021980286),
 (&#39;할아버지&#39;, 0.5890004634857178),
 (&#39;아저씨&#39;, 0.577961802482605),
 (&#39;친구&#39;, 0.5630447864532471),
 (&#39;누나&#39;, 0.559259295463562)]